# Segmenting and Clustering Neighborhoods in Toronto

The objective of this project is to apply a K-means clustering to different neighbourhoods in Toronto, Canada, and extract valuable insights through this clustering. We do not have an existing dataset containing all details of neighbourhoods in Toronto so we will need to find a source for this information online. 

## Part 1

In the first part of this project, the objective is to extract strings representing neighbourhood and their respective details. To accomplish this extraction, Wikipedia has a useful table with the precise details needed. First, the html parsing was executed through the requests library and then processed by beautifulsoup which has many useful features to manipulate documents with html format. 

A for loop was constructed in order to parse through cells related to each neighbourhood. Once this foor loop is done, we drop every row (which represent every neighbourhood) that have 'Not assigned' values in the District column as they do not contain any valuable information. 

In [59]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 

# More rows and columns to visualize
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 5000)

In [150]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.content, 'html.parser')

# Find the table with the list of postal codes
table = soup.find('table')
table_rows = table.find_all("tr")


# for loop through rows of the table and extract each cell from a particular row as a string
def row_string_parser(table_rows):
    rows = []
    for tr in table_rows:
        temp = []
        for row in tr:
            if len(str(row)) > 1:
                string = str(row)[4:-6]
                temp.append(string)
        rows.append(list(temp))
        
    return rows


rows = row_string_parser(table_rows)
# Create dataframe
dataframe = pd.DataFrame(rows)

# Convert first row into dataframe columns
dataframe.columns = dataframe.iloc[0]
dataframe = dataframe[1:]
print(dataframe.head())
print(dataframe.shape)

# Data imputation for rows with values 'Not assigned'
dataframe = dataframe[dataframe['District'] != 'Not assigned']
dataframe['Neighbourhood'] = np.where(dataframe['Neighbourhood'] == 'Not assigned', dataframe['District'] , dataframe['Neighbourhood'])
dataframe = dataframe.reset_index(drop=True)
display(dataframe.head())

print('Final shape of the dataframe:')
display(dataframe.shape)

0 Postal Code          District              Neighbourhood
1         M1A      Not assigned               Not assigned
2         M2A      Not assigned               Not assigned
3         M3A        North York                  Parkwoods
4         M4A        North York           Victoria Village
5         M5A  Downtown Toronto  Regent Park, Harbourfront
(180, 3)


,Postal Code,District,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Final shape of the dataframe:


(103, 3)

## Part 2

The python library "geocoder" claims to offers a free software that extracts coordinates given a postal code. In the case of postal codes from Toronto, Canada, this software failed to work. A while loop was implemented to avoid a bug in the geocoder api which returns a none value. Nevertheless, the geocoder returned none values no matter how much iterations it run in the while loop. For this reason, I opted to use the csv file which already contains all coordinates from the postal codes in  Toronto. Once loaded the csv file with the postal codes and converted to a dataframe, I merged it with the previous dataframe. The function pandas.merge works simlar to the SQL join clause which gives the flexibility to merge two datasets through id's in different ways (right, left). 

In [153]:
coordinates = pd.read_csv(r'C:\Users\adminit\Data Science Projects\Geospatial_Coordinates.csv')
display(coordinates.head())

dataframe = pd.merge(left=dataframe, right=coordinates, left_on='Postal Code', right_on='Postal Code')
display(dataframe.head())

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


,Postal Code,District,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Part 3 

Now that we have all postal codes for Toronto, Canada, we can create a map and use the coordinates as markers for every post code. Folium is a reliable, easy and dynamic python library which helps us map detailed geographical representations of the coordinates we want to analyze. 

In [154]:
# import libraries for geograpical mapping 
from geopy.geocoders import Nominatim 
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent=)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [155]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district, neighborhood in zip(dataframe['Latitude'], dataframe['Longitude'], dataframe['District'], dataframe['Neighbourhood']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now we will cluster neighbourhoods by their popular venues. In this case, I will focus the analysis on the district which has the highest amount of neighbourhoods. This selection could be made through pandas' groupby operation and then applying a count method. Once we have the districts in descending order, we create another dataframe which contains all neighbourhoods of the most popular district, in this case, North York. 

In [261]:
popular_districts = dataframe.groupby('District').count().sort_values('Postal Code', ascending=False)
print(popular_districts)
popular_district = dataframe[dataframe['District'] == popular_districts.index.values[0]]
north_york_data = popular_district.drop('District', 1)

print(north_york_data)

                  Postal Code  Neighbourhood  Latitude  Longitude
District                                                         
North York                 24             24        24         24
Downtown Toronto           19             19        19         19
Scarborough                17             17        17         17
Etobicoke                  12             12        12         12
Central Toronto             9              9         9          9
West Toronto                6              6         6          6
East Toronto                5              5         5          5
East York                   5              5         5          5
York                        5              5         5          5
Mississauga                 1              1         1          1
   Postal Code                                    Neighbourhood   Latitude  Longitude
0          M3A                                        Parkwoods  43.753259 -79.329656
1          M4A                      

In a similar way of how we mapped Toronto, now we will map the North York district.

In [176]:
address = 'North York, Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location_northyk = geolocator.geocode(address)
latitude_northyk = location_northyk.latitude
longitude_northyk = location_northyk.longitude
print('The geograpical coordinates of North York are {}, {}.'.format(latitude_northyk, longitude_northyk))

The geograpical coordinates of North York are 43.7543263, -79.44911696639593.


In [179]:
map_northyork = folium.Map(location=[latitude_northyk, longitude_northyk], zoom_start=12)

for lat, lng, label in zip(north_york_data['Latitude'], north_york_data['Longitude'], north_york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_northyork)
    
map_northyork

In [ ]:
CLIENT_ID = 'client_id' # your Foursquare ID
CLIENT_SECRET = 'client_secret' # your Foursquare Secret
VERSION = '20200828' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [187]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            50)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [188]:
northyork_venues = getNearbyVenues(names=north_york_data['Neighbourhood'],
                                   latitudes=north_york_data['Latitude'],
                                   longitudes=north_york_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [202]:
display(northyork_venues.shape)

(223, 7)

In [262]:
display(northyork_venues.head())

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [209]:
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 94 uniques categories.


In [210]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.tail()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Snack Place,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
218,"Willowdale, Willowdale West",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
219,"Willowdale, Willowdale West",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
220,"Willowdale, Willowdale West",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
221,"Willowdale, Willowdale West",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
222,"Willowdale, Willowdale West",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [211]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()

northyork_grouped


,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Snack Place,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.090909,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.045455,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.090909,0.000000,0.00,0.000000,0.00,0.045455,0.00,0.000000,0.045455,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.045455,0.045455,0.000000,0.045455,0.0,0.000000,0.045455,0.000000,0.0,0.000000,0.045455,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.045455,0.000000,0.045455,0.000000,0.045455,0.000000,0.045455,0.045455,0.000000,0.0,0.0,0.000000,0.000000,0.045455,0.00,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.250000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.045455,0.045455,0.000000,0.000000,0.00,0.000000,0.090909,0.045455,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.045455,0.045455,0.000000,0.0,0.000000,0.000000,0.045455,0.000000,0.0,0.090909,0.000000,0.045455,0.045455,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.0,0.0,0.045455,0.000000,0.045455,0.00,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.00,0.045455,0.00,0.045455,0.00,0.000000,0.00
3,Don Mills,0.000000,0.000000,0.000000,0.037037,0.000000,0.037037,0.037037,0.00,0.000000,0.000000,0.037037,0.00,0.074074,0.037037,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.037037,0.037037,0.037037,0.00,0.037037,0.074074,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.037037,0.00000

In [215]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0    Coffee Shop  0.09
1           Bank  0.09
2  Shopping Mall  0.05
3  Grocery Store  0.05
4    Gas Station  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.09
1      Sandwich Place  0.09
2  Italian Restaurant  0.09
3    Greek Restaurant  0.05
4   Indian Restaurant  0.05


----Don Mills----
                 venue  freq
0                  Gym  0.11
1          Coffee Shop  0.07
2           Restaurant  0.07
3  Japanese Restaurant  0.07
4           Beer Store  0.07


----Downsview----
                venue  freq
0       Grocery Store  0.20
1                Park  0.13
2               Hotel  0.07
3  Athletics & Sports  0.07
4                Bank  0.07


----Fairvi

In [259]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(19, 11)

In [366]:
# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([3, 3, 3, 3, 3, 3, 3, 3, 4, 1, 3, 0, 3, 0, 3, 2, 3, 3, 2])

In [367]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


northyork_merged = north_york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood').dropna()
northyork_merged['Cluster Labels'] = northyork_merged['Cluster Labels'].astype(int)

northyork_merged # check the last columns!

,Postal Code,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,M4A,Victoria Village,43.725882,-79.315572,3,Intersection,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,M6A,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3,Clothing Store,Accessories Store,Boutique,Event Space,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Athletics & Sports,Bakery,Convenience Store
7,M3B,Don Mills,43.745906,-79.352188,3,Gym,Restaurant,Coffee Shop,Beer Store,Japanese Restaurant,Bike Shop,Baseball Field,Dim Sum Restaurant,Café,Discount Store
10,M6B,Glencairn,43.709577,-79.445073,3,Pizza Place,Asian Restaurant,Sushi Restaurant,Pub,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
13,M3C,Don Mills,43.725900,-79.340923,3,Gym,Restaurant,Coffee Shop,Beer Store,Japanese Restaurant,Bike Shop,Baseball Field,Dim Sum Restaurant,Café,Discount Store
27,M2H,Hillcrest Village,43.803762,-79.363452,3,Golf Course,Pool,Athletics & Sports,Mediterranean Restaurant,Dog Run,Women's Store,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
28,M3H,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,3,Coffee Shop,Bank,Gas Station,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Grocery Store,Deli / Bodega,Ice Cream Shop
33,M2J,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,3,Clothing Store,Coffee Shop,Fast Food Restaurant,Bank,Juice Bar,Restaurant,Women's Store,Toy / Game Store,Bar,Boutique
34,M3J,"Northwood Park, York University",43.767980,-79.487262,3,Furniture / Home Store,Caribbean Restaurant,Miscellaneous Shop,Falafel Restaurant,Massage Studio,Bar,Coffee Shop,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping


In [368]:
# create map
map_clusters = folium.Map(location=[latitude_northyk, longitude_northyk], zoom_start=11)

# set color scheme for the clusters
rainbow = ['red', 'blue', 'orange', 'darkred', 'purple']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighbourhood'], northyork_merged['Cluster Labels']):
    print(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

0
3
3
3
3
3
3
3
3
3
3
3
3
0
4
2
3
3
1
3
3
2
3


## Clusters 

There seems to be a problem with the lack of features for the K-means clustering. As a result, the K-means classifies outliers as an individual category. For example, the data point in cluster 5 may have been clasified as a separate category for having "chocolate shop" as one of its popular venues (absent in any other neighbourhood). Adding more feature, such as tips and menus, could enhance clustering but in this case we would require to pay the premium membership in foursquare.

## Cluster 1 

In [374]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Park,Food & Drink Shop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
49,"North Park, Maple Leaf Park, Upwood Park",Basketball Court,Construction & Landscaping,Bakery,Park,Discount Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega


## Cluster 2

In [370]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,"Humberlea, Emery",Baseball Field,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


## Cluster 3

In [371]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,"Willowdale, Newtonbrook",Park,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
66,York Mills West,Park,Convenience Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


## Cluster 4

In [372]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Intersection,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Accessories Store,Boutique,Event Space,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Athletics & Sports,Bakery,Convenience Store
7,Don Mills,Gym,Restaurant,Coffee Shop,Beer Store,Japanese Restaurant,Bike Shop,Baseball Field,Dim Sum Restaurant,Café,Discount Store
10,Glencairn,Pizza Place,Asian Restaurant,Sushi Restaurant,Pub,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
13,Don Mills,Gym,Restaurant,Coffee Shop,Beer Store,Japanese Restaurant,Bike Shop,Baseball Field,Dim Sum Restaurant,Café,Discount Store
27,Hillcrest Village,Golf Course,Pool,Athletics & Sports,Mediterranean Restaurant,Dog Run,Women's Store,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
28,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gas Station,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Grocery Store,Deli / Bodega,Ice Cream Shop
33,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Bank,Juice Bar,Restaurant,Women's Store,Toy / Game Store,Bar,Boutique
34,"Northwood Park, York University",Furniture / Home Store,Caribbean Restaurant,Miscellaneous Shop,Falafel Restaurant,Massage Studio,Bar,Coffee Shop,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping
39,Bayview Village,Chinese Restaurant,Japanese Restaurant,Café,Bank,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


## Cluster 5

In [373]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Humber Summit,Furniture / Home Store,Pizza Place,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
